Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [27]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [28]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [29]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [30]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

L2正则化

In [31]:
def forward_propagation(features, weights, biases):
  hidden_layer = tf.add(tf.matmul(features, weights[0]), biases[0])
  hidden_layer = tf.nn.relu(hidden_layer)    
  logits = tf.matmul(hidden_layer, weights[1]) + biases[1]
  return logits


batch_size = 128
hidden_nodes_num = 1024
reglarziation_rate = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.

  hidden_layer_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  hidden_layer_biases = tf.Variable(tf.zeros([hidden_nodes_num]))

  out_weights = tf.Variable(
    tf.truncated_normal([hidden_nodes_num, num_labels]))
  out_biases = tf.Variable(tf.zeros([num_labels]))

  # Weights and biases
  weights = [hidden_layer_weights, out_weights]
  biases = [hidden_layer_biases, out_biases]
  
  # Training computation.
  logits = forward_propagation(tf_train_dataset, weights, biases)
  # key code: L2 regularization
  l2_loss = tf.nn.l2_loss(weights[0]) + tf.nn.l2_loss(weights[1])  
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    ) + reglarziation_rate * l2_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_propagation(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(forward_propagation(tf_test_dataset, weights, biases))

In [32]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3633.210693
Minibatch accuracy: 7.8%
Validation accuracy: 32.9%
Minibatch loss at step 500: 21.249147
Minibatch accuracy: 86.7%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.789106
Minibatch accuracy: 85.2%
Validation accuracy: 83.7%
Minibatch loss at step 1500: 0.791339
Minibatch accuracy: 80.5%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 0.722039
Minibatch accuracy: 86.7%
Validation accuracy: 83.4%
Minibatch loss at step 2500: 0.602262
Minibatch accuracy: 88.3%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 0.691497
Minibatch accuracy: 86.7%
Validation accuracy: 83.3%
Test accuracy: 89.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

使用很少batches训练，会使得模型在training data上准确度非常高，甚至达到100%，但是在valid data和test data表现不会太好

In [33]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # Restrict your training data to just a few batches. 
    tmp_step = step % 5
    offset = (tmp_step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3501.808594
Minibatch accuracy: 13.3%
Validation accuracy: 23.4%
Minibatch loss at step 500: 21.137032
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Minibatch loss at step 1000: 0.536494
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 1500: 0.367460
Minibatch accuracy: 100.0%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 0.354722
Minibatch accuracy: 100.0%
Validation accuracy: 79.5%
Minibatch loss at step 2500: 0.347790
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 3000: 0.343715
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Test accuracy: 86.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [34]:
def forward_propagation(features, weights, biases):
  hidden_layer = tf.add(tf.matmul(features, weights[0]), biases[0])
  hidden_layer = tf.nn.relu(hidden_layer)    
  logits = tf.matmul(hidden_layer, weights[1]) + biases[1]
  return logits


def forward_propagation_with_dropout(features, weights, biases):
  hidden_layer = tf.add(tf.matmul(features, weights[0]), biases[0])
  hidden_layer = tf.nn.dropout(tf.nn.relu(hidden_layer), keep_prob=keep_prob)    
  logits = tf.matmul(hidden_layer, weights[1]) + biases[1]
  return logits

batch_size = 128
hidden_nodes_num = 1024
keep_prob = 0.9
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that wibll be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.

  hidden_layer_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_num]))
  hidden_layer_biases = tf.Variable(tf.zeros([hidden_nodes_num]))

  out_weights = tf.Variable(
    tf.truncated_normal([hidden_nodes_num, num_labels]))
  out_biases = tf.Variable(tf.zeros([num_labels]))

  # Weights and biases
  weights = [hidden_layer_weights, out_weights]
  biases = [hidden_layer_biases, out_biases]
  
  # Training computation. Dropout
  logits = forward_propagation_with_dropout(tf_train_dataset, weights, biases)
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)
    ) 
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_propagation(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(forward_propagation(tf_test_dataset, weights, biases))

In [35]:
num_steps = 3001
print(train_labels.shape[0])
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    # Restrict your training data to just a few batches. 
    # tmp_step = step % 5
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

200000
Initialized
Minibatch loss at step 0: 323.325012
Minibatch accuracy: 12.5%
Validation accuracy: 35.5%
Minibatch loss at step 500: 10.583594
Minibatch accuracy: 78.9%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 17.918354
Minibatch accuracy: 74.2%
Validation accuracy: 79.7%
Minibatch loss at step 1500: 5.107863
Minibatch accuracy: 77.3%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 2.847394
Minibatch accuracy: 79.7%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 4.260151
Minibatch accuracy: 84.4%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 5.177354
Minibatch accuracy: 83.6%
Validation accuracy: 81.9%
Test accuracy: 88.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


### Network Architecture 

- input layer: 28 * 28
- hidden layers: 1024-300-50
- output layer: 10
- activation function: relu
- learning rate: linear decay of the learning rate or tf.train.exponential_decay
- dropout keep prob: 0.5
- batchsize:128
- optimizer: tf.train.MomentumOptimizer

In [121]:
import tensorflow as tf

def forward_propagation_with_one_dropout(features, weights, biases):
  
  hidden_layer_1 = tf.add(tf.matmul(features, weights[0]), biases[0])
  hidden_layer_1 = tf.nn.relu(hidden_layer_1)
    
  hidden_layer_2 = tf.add(tf.matmul(hidden_layer_1, weights[1]), biases[1])
  hidden_layer_2 = tf.nn.relu(hidden_layer_2)
    
  hidden_layer_3 = tf.add(tf.matmul(hidden_layer_2, weights[2]), biases[2])
  hidden_layer_3 = tf.nn.dropout(tf.nn.relu(hidden_layer_3), keep_prob=keep_prob)
  
  logits = tf.matmul(hidden_layer_3, weights[-1]) + biases[-1]
  return logits


def forward_propagation(features, weights, biases):
  
  hidden_layer_1 = tf.add(tf.matmul(features, weights[0]), biases[0])
  hidden_layer_1 = tf.nn.relu(hidden_layer_1)
    
  hidden_layer_2 = tf.add(tf.matmul(hidden_layer_1, weights[1]), biases[1])
  hidden_layer_2 = tf.nn.relu(hidden_layer_2)
    
  hidden_layer_3 = tf.add(tf.matmul(hidden_layer_2, weights[2]), biases[2])
  hidden_layer_3 = tf.nn.relu(hidden_layer_3)
  
  logits = tf.matmul(hidden_layer_3, weights[-1]) + biases[-1]
  return logits


def bias_variable(shape):
  """
  由于使用的是ReLU神经元,
  用一个较小的正数来初始化偏置项，以避免神经元节点输出恒为0的问题（dead neurons）
  """
  initial = tf.constant(0., shape=shape)
  return tf.Variable(initial)


def weight_variable(shape):
  """
  权重在初始化时应该加入少量的噪声来打破对称性以及避免0梯度
  """
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

batch_size = 128
hidden_nodes_num = [1024, 512, 30]
keep_prob = 0.5
graph = tf.Graph()

with graph.as_default():

  # Input data. For the training data, we use a placeholder that wibll be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # Weights and biases
  weights = []
  biases = []
  begin_num = image_size * image_size
  
  for i in hidden_nodes_num:
    end_num = i
    weights.append(weight_variable([begin_num, end_num]))
    biases.append(bias_variable([end_num]))
    begin_num = end_num
    
  weights.append(weight_variable([hidden_nodes_num[-1], num_labels]))
  biases.append(bias_variable([num_labels]))

  # Training computation.
  logits = forward_propagation_with_one_dropout(tf_train_dataset, weights, biases)
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  """
  decayed_learning_rate = learning_rate *
                        decay_rate ^ (global_step / decay_steps)
  """

  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, decay_steps=1000, decay_rate=0.7, staircase=True)
  optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.6).minimize(loss, global_step=global_step)
  # optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(forward_propagation(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(forward_propagation(tf_test_dataset, weights, biases))

In [123]:
num_steps = 6001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
    
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.079024
Minibatch accuracy: 10.2%
Validation accuracy: 13.2%
Minibatch loss at step 500: 0.641496
Minibatch accuracy: 77.3%
Validation accuracy: 83.6%
Minibatch loss at step 1000: 0.515654
Minibatch accuracy: 82.8%
Validation accuracy: 85.0%
Minibatch loss at step 1500: 0.609164
Minibatch accuracy: 82.0%
Validation accuracy: 86.2%
Minibatch loss at step 2000: 0.375060
Minibatch accuracy: 88.3%
Validation accuracy: 86.9%
Minibatch loss at step 2500: 0.373803
Minibatch accuracy: 89.1%
Validation accuracy: 87.3%
Minibatch loss at step 3000: 0.443877
Minibatch accuracy: 87.5%
Validation accuracy: 87.6%
Minibatch loss at step 3500: 0.387128
Minibatch accuracy: 84.4%
Validation accuracy: 87.7%
Minibatch loss at step 4000: 0.398968
Minibatch accuracy: 87.5%
Validation accuracy: 87.9%
Minibatch loss at step 4500: 0.528143
Minibatch accuracy: 86.7%
Validation accuracy: 88.0%
Minibatch loss at step 5000: 0.500311
Minibatch accuracy: 83.6%
Validation accurac